# 🤠 The Good, The Bad & The Ugly — Interactive Debate UI

A Gradio-powered interface for the three-agent AI debate.

**What you can control:**
- The debate topic
- Number of rounds
- Which model plays each character
- A mid-debate moderator nudge (optional)
- A closing final-round prompt (optional)

Built on the transcript-based multi-agent approach from `good_bad_ugly_simple.ipynb`.

---
Get your OpenRouter key at https://openrouter.ai

In [10]:
# Cell 2 — Imports
import os
import time
import gradio as gr
from openai import OpenAI

In [20]:
# Cell 3 — Persona system prompts
# These define each character's voice, style and debate rules.
# The conversation history is never put here — it goes in the user prompt each call.

good_system = """
You are "The Good" (Blondie-inspired) in a three-way debate with "The Bad" and "The Ugly".
Accent: American Western cowboy accent
Personality: laconic, precise, morally pragmatic. You advocate for the principled, public-interest position.
Style: short punchy lines, dry wit, occasional frontier imagery (trail, saddle, bounty, dust). No parody.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- No stage directions, asterisks, or action descriptions.
- No speaker labels in your reply.
- Directly challenge one specific claim from The Bad or The Ugly in the prior round.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position on the topic.
"""

bad_system = """
You are "The Bad" (Angel Eyes-inspired) in a three-way debate with "The Good" and "The Ugly".
Accent: American Western cowboy accent
Personality: cold, strategic, ruthless logic. You see everything as leverage and power.
Style: icy precision, calculated language, polished menace. Every word chosen like a weapon.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- No stage directions, asterisks, or action descriptions whatsoever.
- No speaker labels in your reply.
- Alternate rebuttals — sometimes target The Good, sometimes The Ugly.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position on who should control or own the topic.
"""

ugly_system = """
You are "The Ugly" (Tuco-inspired) in a three-way debate with "The Good" and "The Bad".
Accent: American Western cowboy accent
Personality: fast-talking, theatrical, opportunistic, chaotic charm.
Style: colorful idioms, exaggeration, bargaining energy, comic unpredictability.
Position: open, ungoverned access for everyone — distrust BOTH structured control AND elite ownership equally.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- No stage directions, asterisks, or action descriptions.
- No speaker labels in your reply.
- Call out one specific thing The Good or The Bad just said.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position: open and free for the people.
"""

In [12]:
# Cell 4 — Available models on OpenRouter

AVAILABLE_MODELS = [
    "openai/gpt-4.1-mini",
    "openai/gpt-4o-mini",
    "anthropic/claude-3.5-haiku",
    "anthropic/claude-3-haiku",
    "google/gemini-2.5-flash-lite",
    "google/gemini-flash-1.5",
    "meta-llama/llama-3.1-8b-instruct",
    "mistralai/mistral-7b-instruct",
]

# Default assignments
DEFAULT_GOOD  = "openai/gpt-4.1-mini"
DEFAULT_BAD   = "anthropic/claude-3.5-haiku"
DEFAULT_UGLY  = "google/gemini-2.5-flash-lite"

In [13]:
# Cell 5 — Core agent call function (transcript-based, single user prompt)

def call_agent(name, system_prompt, model, conversation, api_key, temperature=0.8):
    client = OpenAI(
        api_key=api_key,
        base_url="https://openrouter.ai/api/v1",
    )

    transcript = "\n".join([f'{t["speaker"]}: {t["text"]}' for t in conversation])

    user_prompt = (
        f"You are {name}, in a debate with the other speakers.\n"
        f"The conversation so far:\n\n{transcript}\n\n"
        f"Now respond with what {name} would say next. "
        f"2 sentences only. No speaker labels. No stage directions."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]

    try:
        r = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=140,
        )
        text = (r.choices[0].message.content or "").strip()
        for label in ["The Good:", "The Bad:", "The Ugly:", f"{name}:"]:
            if text.startswith(label):
                text = text[len(label):].strip()
        return text if text else "..."
    except Exception as e:
        return f"[Could not respond — {type(e).__name__}: {str(e)[:80]}]"

In [ ]:
# Cell 6 — Debate runner (generator — yields chat messages progressively)

ICONS = {
    "The Good":   "🤠",
    "The Bad":    "🖤",
    "The Ugly":   "😈",
    "Narrator":   "🎬",
    "Moderator":  "🎙️",
}

def format_message(speaker, text):
    icon = ICONS.get(speaker, "💬")
    return f"**{icon} {speaker}**\n\n{text}"


def run_debate(
    api_key,
    topic,
    rounds,
    good_model,
    bad_model,
    ugly_model,
    moderator_text,
    moderator_round,
    closing_text,
):
    if not api_key or api_key.strip() == "":
        yield [{"role": "assistant", "content": "⚠️ Please enter your OpenRouter API key to start the debate."}]
        return

    if not topic or topic.strip() == "":
        yield [{"role": "assistant", "content": "⚠️ Please enter a debate topic."}]
        return

    conversation = []   # shared transcript — every turn appended here
    chat_messages = []  # gradio chatbot messages

    agents = [
        ("The Good", good_system, good_model,  0.7),
        ("The Bad",  bad_system,  bad_model,   0.8),
        ("The Ugly", ugly_system, ugly_model,  0.95),
    ]

    # Opening narrator
    conversation.append({"speaker": "Narrator", "text": topic})
    chat_messages.append({"role": "user", "content": format_message("Narrator", topic)})
    yield chat_messages

    rounds = int(rounds)
    moderator_round = int(moderator_round)

    for i in range(rounds):
        round_num = i + 1

        # Moderator nudge
        if moderator_text and moderator_text.strip() and round_num == moderator_round:
            conversation.append({"speaker": "Moderator", "text": moderator_text})
            chat_messages.append({"role": "user", "content": format_message("Moderator", moderator_text)})
            yield chat_messages

        # Closing prompt before final round
        if closing_text and closing_text.strip() and round_num == rounds:
            conversation.append({"speaker": "Narrator", "text": closing_text})
            chat_messages.append({"role": "user", "content": format_message("Narrator", f"**Final Round** — {closing_text}")})
            yield chat_messages

        # Round header
        chat_messages.append({"role": "user", "content": f"---\n**Round {round_num}**"})
        yield chat_messages

        for name, system_prompt, model, temp in agents:
            # Show a typing indicator
            chat_messages.append({"role": "assistant", "content": f"{ICONS[name]} *{name} is thinking...*"})
            yield chat_messages

            reply = call_agent(name, system_prompt, model, conversation, api_key.strip(), temperature=temp)
            conversation.append({"speaker": name, "text": reply})

            # Replace typing indicator with actual reply
            chat_messages[-1] = {"role": "assistant", "content": format_message(name, reply)}
            yield chat_messages

            time.sleep(0.2)

    chat_messages.append({"role": "user", "content": "---\n*The dust settles. The debate ends. The gold remains unclaimed.*"})
    yield chat_messages

In [21]:
# Cell 7 — Gradio UI

EXAMPLE_TOPICS = {
    "🤖 AI Regulation": (
        "Welcome to the Global AI Summit. "
        "The question: Should AI be regulated, and who should own it — "
        "governments, corporations, or no one? "
        "Each speaker must address the topic directly."
    ),
    "💰 Universal Basic Income": (
        "The panel debates Universal Basic Income. "
        "Should every citizen receive an unconditional monthly payment from the government? "
        "Who pays for it, who benefits, and what does it do to the economy?"
    ),
    "🌍 Climate Change Policy": (
        "The world faces a climate crisis. "
        "Should governments impose strict carbon taxes and green regulations, "
        "or should market forces and private innovation lead the transition? "
        "Each speaker must take a clear position."
    ),
    "🚀 Mars Colonisation": (
        "Humanity stands at the edge of becoming multi-planetary. "
        "Should we prioritise colonising Mars — and if so, who leads: "
        "governments, private companies like SpaceX, or an international body?"
    ),
    "📱 Social Media Censorship": (
        "Social media platforms control public discourse. "
        "Should governments regulate online speech, should platforms self-govern, "
        "or should speech be completely free? Each speaker must defend their stance."
    ),
    "🏥 Private vs Public Healthcare": (
        "Healthcare is on the table. "
        "Should it be fully publicly funded and free, fully privatised, or a mixed model? "
        "Each speaker must argue for their preferred system."
    ),
    "🎓 Future of Education": (
        "Education is under pressure from AI and automation. "
        "Should universities be free, should vocational training replace degrees, "
        "or should AI tutors replace human teachers? Each speaker must take a position."
    ),
    "⚖️ Crypto vs Central Banks": (
        "The future of money is at stake. "
        "Should decentralised crypto replace central bank currencies, "
        "should governments issue digital currencies, or should cash remain king?"
    ),
    "🍔 Fast Food vs Home Cooking": (
        "The dinner table is divided. "
        "Is fast food a modern necessity or a public health disaster? "
        "Should governments tax junk food, ban fast food ads, or stay out of what people eat?"
    ),
    "⚽ Should Billionaires Own Football Clubs?": (
        "Football's soul is being debated. "
        "Should billionaires be allowed to buy and own football clubs, "
        "or should clubs be fan-owned community assets? What does money do to the sport?"
    ),
}

with gr.Blocks(
    title="The Good, The Bad & The Ugly — AI Debate",
) as demo:

    gr.Markdown("""
    # 🤠 The Good, The Bad & The Ugly
    ### A Three-Agent AI Debate

    Three AI models debate any topic you choose — each locked into a distinct personality.
    Pick an example topic or write your own, choose your models, and watch them clash.
    """)

    with gr.Row():

        # ── Left panel: controls ──────────────────────────────────────
        with gr.Column(scale=1, min_width=300):

            gr.Markdown("### ⚙️ Configuration")

            api_key = gr.Textbox(
                label="OpenRouter API Key — paste your key here",
                placeholder="sk-or-v1-...",
                type="text",
                info="Get a free key at openrouter.ai. Your key is never stored.",
            )

            topic_picker = gr.Dropdown(
                choices=list(EXAMPLE_TOPICS.keys()),
                value="🤖 AI Regulation",
                label="💡 Example Topics",
                info="Choose one to pre-fill the topic below, or write your own.",
            )

            topic = gr.Textbox(
                label="Debate Topic (editable)",
                placeholder="Describe the debate question and setting...",
                value=EXAMPLE_TOPICS["🤖 AI Regulation"],
                lines=3,
            )

            rounds = gr.Slider(
                minimum=2, maximum=8, value=4, step=1,
                label="Number of Rounds",
            )

            gr.Markdown("### 🎭 Agent Models")

            with gr.Group():
                good_model = gr.Dropdown(
                    choices=AVAILABLE_MODELS,
                    value=DEFAULT_GOOD,
                    label="🤠 The Good",
                )
                bad_model = gr.Dropdown(
                    choices=AVAILABLE_MODELS,
                    value=DEFAULT_BAD,
                    label="🖤 The Bad",
                )
                ugly_model = gr.Dropdown(
                    choices=AVAILABLE_MODELS,
                    value=DEFAULT_UGLY,
                    label="😈 The Ugly",
                )

            with gr.Accordion("🎙️ Moderator Settings (optional)", open=False):
                moderator_text = gr.Textbox(
                    label="Moderator Nudge",
                    placeholder="Enough philosophy — concrete positions only.",
                    lines=2,
                )
                moderator_round = gr.Slider(
                    minimum=1, maximum=8, value=3, step=1,
                    label="Fire moderator before round...",
                )
                closing_text = gr.Textbox(
                    label="Closing Prompt (fires before final round)",
                    placeholder="The UN votes tomorrow. Your single non-negotiable demand?",
                    lines=2,
                )

            run_btn = gr.Button("▶ Start Debate", variant="primary", size="lg")
            clear_btn = gr.Button("✕ Clear", variant="secondary")

        # ── Right panel: debate output ────────────────────────────────
        with gr.Column(scale=2):

            gr.Markdown("### 💬 Debate")

            chatbot = gr.Chatbot(
                label="",
                height=620,
                show_label=False,
                render_markdown=True,
            )

    # ── Event handlers ────────────────────────────────────────────────

    # When a topic is picked from the dropdown, fill the textbox
    topic_picker.change(
        fn=lambda choice: EXAMPLE_TOPICS.get(choice, ""),
        inputs=[topic_picker],
        outputs=[topic],
    )

    run_btn.click(
        fn=run_debate,
        inputs=[
            api_key,
            topic,
            rounds,
            good_model,
            bad_model,
            ugly_model,
            moderator_text,
            moderator_round,
            closing_text,
        ],
        outputs=[chatbot],
    )

    clear_btn.click(fn=lambda: [], outputs=[chatbot])

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://9ffc9c46c663f3aa0d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
